In [1]:
import os
import sys
sys.path.append("../../../../")
os.environ["TOKENIZERS_PARALLELISM"] = "false"

In [2]:
import copy
import torch
from datetime import datetime
from utils.helper import ModelConfig, color_print
from utils.dataset_utils.load_dataset import (
    load_data,
)
from utils.model_utils.save_module import save_module
from utils.model_utils.load_model import load_model
from utils.model_utils.evaluate import evaluate_model, get_sparsity, similar
from utils.dataset_utils.sampling import SamplingDataset
from utils.prune_utils.prune import (
    prune_magnitude
)

In [3]:
name= "IMDB"
device = torch.device("cuda:1")
checkpoint = None
batch_size=32
num_workers=48
num_samples=16
magnitude_ratio=0.6
seed=44
include_layers=["attention", "intermediate", "output"]
exclude_layers=None

In [4]:
script_start_time = datetime.now()
print(f"Script started at: {script_start_time.strftime('%Y-%m-%d %H:%M:%S')}")

Script started at: 2024-08-19 22:28:50


In [5]:
model_config = ModelConfig(name, device)
num_labels = model_config.config["num_labels"]
model, tokenizer, checkpoint = load_model(model_config)

Loading the model.

{'model_name': 'textattack/bert-base-uncased-imdb', 'task_type': 'classification', 'architectures': 'bert', 'dataset_name': 'IMDB', 'num_labels': 2, 'cache_dir': 'Models'}

The model textattack/bert-base-uncased-imdb is loaded.

In [6]:
train_dataloader, valid_dataloader, test_dataloader = load_data(
    name, batch_size=batch_size, num_workers=num_workers, seed=seed
)

{'dataset_name': 'IMDB', 'path': 'imdb', 'config_name': 'plain_text', 'text_column': 'text', 'label_column': 'label', 'cache_dir': 'Datasets/IMDB', 'task_type': 'classification'}

Loading cached dataset IMDB.

The dataset IMDB is loaded

In [7]:
all_samples = SamplingDataset(
    train_dataloader, 200, num_samples, num_labels, False, 4, device=device, resample=False, seed=seed
)

In [8]:
# print("Evaluate the original model")
# result = evaluate_model(model, model_config, test_dataloader)

In [9]:
# Evaluate the original model
# Evaluating: 100%|█████████████████████████████████████████████████████████████████████| 782/782 [05:36<00:00,  2.32it/s]
# Loss: 0.3423
# Precision: 0.9306, Recall: 0.9303, F1-Score: 0.9303
#               precision    recall  f1-score   support

#            0       0.92      0.94      0.93     12500
#            1       0.94      0.92      0.93     12500

#     accuracy                           0.93     25000
#    macro avg       0.93      0.93      0.93     25000
# weighted avg       0.93      0.93      0.93     25000

In [10]:
module = copy.deepcopy(model)
prune_magnitude(module, sparsity_ratio=magnitude_ratio, include_layers=include_layers, exclude_layers=exclude_layers)
print("Evaluate the pruned model")
result = evaluate_model(model, model_config, test_dataloader)
# save_module(module, "Modules/", f"magnitude_{name}_{magnitude_ratio}p.pt")

Evaluate the pruned model

Evaluating:   0%|          | 0/782 [00:00<?, ?it/s]

Evaluating:   0%|          | 1/782 [00:00<07:11,  1.81it/s]

Evaluating:   0%|          | 2/782 [00:00<04:37,  2.81it/s]

Evaluating:   0%|          | 3/782 [00:00<03:48,  3.41it/s]

...

Evaluating: 100%|██████████| 782/782 [07:21<00:00,  2.12it/s]

Evaluating: 100%|██████████| 782/782 [07:21<00:00,  1.77it/s]

Loss: 0.3423

Precision: 0.9306, Recall: 0.9303, F1-Score: 0.9303

              precision    recall  f1-score   support

           0       0.92      0.94      0.93     12500
           1       0.94      0.92      0.93     12500

    accuracy                           0.93     25000
   macro avg       0.93      0.93      0.93     25000
weighted avg       0.93      0.93      0.93     25000


In [11]:
for concern in range(num_labels):
    print(f"--{concern}--")
    positive_samples = SamplingDataset(
        train_dataloader, concern, num_samples, num_labels, True, 4, device=device, resample=False, seed=seed
    )
    negative_samples = SamplingDataset(
        train_dataloader, concern, num_samples, num_labels, False, 4, device=device, resample=False, seed=seed
    )
    similar(model, module, valid_dataloader, concern, num_samples, num_labels, device=device, seed=seed)

--0--

adding eps to diagonal and taking inverse

taking square root

dot products...

trying to take final svd

...

CCA coefficients mean non-concern: (0.526825608983391, 0.526825608983391)

Linear CKA concern: 0.21492304187486694

Linear CKA non-concern: 0.45374185815892765

Kernel CKA concern: 0.13682594189277977

Kernel CKA non-concern: 0.3756021929163487

In [12]:
get_sparsity(module)

(0.5952683065347829,
 {'bert.encoder.layer.0.attention.self.query.weight': 0.5999993218315972,
  'bert.encoder.layer.0.attention.self.query.bias': 0.0,
  'bert.encoder.layer.0.attention.self.key.weight': 0.5999993218315972,
  'bert.encoder.layer.0.attention.self.key.bias': 0.0,
  'bert.encoder.layer.0.attention.self.value.weight': 0.5999993218315972,
  'bert.encoder.layer.0.attention.self.value.bias': 0.0,
  'bert.encoder.layer.0.attention.output.dense.weight': 0.5999993218315972,
  'bert.encoder.layer.0.attention.output.dense.bias': 0.0,
  'bert.encoder.layer.0.intermediate.dense.weight': 0.599999745686849,
  'bert.encoder.layer.0.intermediate.dense.bias': 0.0,
  'bert.encoder.layer.0.output.dense.weight': 0.599999745686849,
  'bert.encoder.layer.0.output.dense.bias': 0.0,
  'bert.encoder.layer.1.attention.self.query.weight': 0.5999993218315972,
  'bert.encoder.layer.1.attention.self.query.bias': 0.0,
  'bert.encoder.layer.1.attention.self.key.weight': 0.5999993218315972,
  'bert.enco